In [ ]:
import pandas as pd
import numpy as np

urop_path = '/home/gridsan/afogelson/osfm/scripts/urop_samples/uniform_sample/uniform_urop_sample_labeled_with_binary.csv'
df = pd.read_csv(urop_path)
df = df[~df['urop'].isna()]
df['urop'] = df['urop'].apply(str.lower)
df = df[df['urop'] != 'select']

In [ ]:
from typing import List
from collections import Counter

def mixture_of_experts(labels: List[str], weights: List[int]):
    weighted_labels = []
    for weight, label in zip(weights, labels):
        weighted_labels += [label for _ in range(weight)]
        
    counter = Counter(weighted_labels)
    return counter.most_common(1)[0][0]
        

In [ ]:
from itertools import product




In [ ]:
def generate_statistics(y_true, y_pred): 
    y_true, y_pred = y_true.apply(str.lower), y_pred.apply(str.lower)
    
    extends_fp = ((y_pred == 'extends') & (y_true != 'extends')).sum() / (y_pred == 'extends').sum()
    extends_fn = ((y_pred != 'extends') & (y_true == 'extends')).sum() / (y_true == 'extends').sum()
    uses_fp = ((y_pred == 'uses') & (y_true != 'uses')).sum() / (y_pred == 'uses').sum()
    uses_fn = ((y_pred != 'uses') & (y_true == 'uses')).sum() / (y_true == 'uses').sum()
    
    return uses_fp, uses_fn, extends_fp, extends_fn

generate_statistics(df['urop'], df['mcllm'])


(0.6025641025641025,
 0.40384615384615385,
 0.5862068965517241,
 0.7142857142857143)

In [ ]:
df.columns

Index(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'sentence', 'mcllm',
       'modelKey', 'paperId', 'modelId_x', 'mc', 'modelId_y', 'mc_reduced',
       'urop_sentence', 'urop', 'urop_assignee', 'gpt', 'mcllmr',
       'mcllm_tuned', 'mcllm_tuned_binary'],
      dtype='object')

In [ ]:
def compute_weighted_stats(df, weights):
    y_true = df['urop']
    y_pred = df.apply(lambda row: mixture_of_experts([row['mc'], row['mcllm'], row['mcllm_tuned'], row['gpt']], weights = weights), axis = 1)
    return generate_statistics(y_true, y_pred)

n = 4
weights_product = list(product(*[list(range(0, n)) for _ in range(n)]))
stats = []
for weights in weights_product: 
    if (sum(weights) == 0):
        continue
    stats.append(compute_weighted_stats(df, weights))


In [ ]:
f_stats = list(map(lambda s: s[0] + s[2], stats))
sorted(list(enumerate(f_stats)), key = lambda s: s[1])

[(24, 0.9392097264437689),
 (25, 0.9392097264437689),
 (29, 0.9392097264437689),
 (30, 0.9392097264437689),
 (44, 0.9392097264437689),
 (45, 0.9392097264437689),
 (46, 0.9392097264437689),
 (92, 0.9762202753441802),
 (105, 0.9801587301587301),
 (110, 0.9801587301587301),
 (122, 0.9801587301587301),
 (125, 0.9801587301587301),
 (126, 0.9801587301587301),
 (190, 0.9801587301587301),
 (93, 0.9812912692589875),
 (3, 0.9859496124031009),
 (4, 0.9859496124031009),
 (7, 0.9859496124031009),
 (8, 0.9859496124031009),
 (9, 0.9859496124031009),
 (11, 0.9859496124031009),
 (12, 0.9859496124031009),
 (13, 0.9859496124031009),
 (14, 0.9859496124031009),
 (23, 0.9859496124031009),
 (27, 0.9859496124031009),
 (28, 0.9859496124031009),
 (43, 0.9859496124031009),
 (71, 0.9859496124031009),
 (75, 0.9859496124031009),
 (76, 0.9859496124031009),
 (91, 0.9859496124031009),
 (139, 0.9859496124031009),
 (21, 0.9930555555555556),
 (26, 0.9930555555555556),
 (38, 0.9930555555555556),
 (42, 0.9930555555555556),

In [ ]:
idx = 106
weights_product[idx], stats[idx]

((1, 2, 2, 2),
 (0.4642857142857143,
  0.4230769230769231,
  0.5344827586206896,
  0.35714285714285715))

In [ ]:
extends_true = df[df['urop'] == 'extends']
extends_true['sentence'].apply(lambda s: s.find('fine') >= 0).sum() / len(extends_true)
extends_true['sentence']

70     the system is built on openai's gpt-2 language...
74     #### 4.1.2 cnn architecture selection\n\n\nthe...
82     the r-cnn [2] uses the imagenet database [22] ...
87     for the second architecture [5], we used an ap...
88     furthermore, to verify the universality of the...
104    to reduce the impact of the framework of model...
105    to demonstrate the effectiveness of our vl-pet...
107    we use robertabase <cite>liu et al (2019)</cit...
113    ##  methodology\n\n\nthis paper focuses on the...
116    with the well-tested encoder-decoder framework...
117    we define the label function \(y_{r}(t_{d}^{c}...
127    our main models, which we refer to as _relbert...
128    ### _experimental setup and implementation det...
129    in this work, we use phobert [24], a monolingu...
136    ##  appendix a hyperparameters for pre-trainin...
137    it can be viewed as an extension of efficientn...
139    the idea is adapted from recent works in neura...
142    ** we fine-tune off-the-